# General description of GSA2PB

## Introduction

GSA2PB have the main objetive to implement a analyse the allosteric communication, based on a structural alphabet, the Protein Blocks (PBs). The model of this program is GSA-tools, developped by Fraternalli *et al.* in 2013 (article), which analyze the trajectories of molecular dynamique by the use of a strucural alphabet. This approach was based on a succession of statistic analyze for predict the allosteric mouvment in a protein. However, GSA-tools use a old version of Gromacs, and was not totally useable.

Here we implement the same statistic analyse but based on the PBs alphabet, which are provide by a free use software : PBxplore.

## Installation:

## I) Select the specific pdb file and compute PBxplore assignement

In the first part of the program, we need to assign the Protein Block alphabet to each pdb file. 

## II) Parsing the fasta file and make it in a right form

After compute the PBassign of PBxplore, we need to import the sequences in the fasta file into Python. To do that, we use the `SeqIO` fonction of `biopython`. We import all the sequences into a `numpy` array, named `fasta_parsed`, with one sequnces in :

In [9]:
from Bio import SeqIO
import numpy as np

def parse_fasta(pathToFile):
    allSeqs = []
    for seq_record in SeqIO.parse(pathToFile, """fasta"""):
            allSeqs.append(seq_record.seq)
    pb_seq = np.array(allSeqs)
    return pb_seq

In [10]:
pathToFile = open("../data/sequences.PB.fasta")

fasta_parsed = parse_fasta(pathToFile)
print(fasta_parsed)

[['Z' 'Z' 'o' ... 'd' 'Z' 'Z']
 ['Z' 'Z' 'o' ... 'd' 'Z' 'Z']
 ['Z' 'Z' 'l' ... 'd' 'Z' 'Z']
 ...
 ['Z' 'Z' 'a' ... 'd' 'Z' 'Z']
 ['Z' 'Z' 'd' ... 'd' 'Z' 'Z']
 ['Z' 'Z' 'b' ... 'd' 'Z' 'Z']]


The fasta file is imported in python but have some defaults: first, we have one frame by rows and columns represent each fragments. For make the statistic analyze more easier, we transpose the matrix.

In [11]:
def transpose_fasta(sequences):
    sequences = sequences.transpose()
    return sequences

fasta_transposed = transpose_fasta(fasta_parsed)
print(fasta_transposed)

[['Z' 'Z' 'Z' ... 'Z' 'Z' 'Z']
 ['Z' 'Z' 'Z' ... 'Z' 'Z' 'Z']
 ['o' 'o' 'l' ... 'a' 'd' 'b']
 ...
 ['d' 'd' 'd' ... 'd' 'd' 'd']
 ['Z' 'Z' 'Z' ... 'Z' 'Z' 'Z']
 ['Z' 'Z' 'Z' ... 'Z' 'Z' 'Z']]


One more default of this array is inherent to the PBxplore : the 'Z' "trajectory". PBxplore analyze the trajectories of each amino acide *n* by the relation with amino acids *n-2*, *n-1*, *n+1*, *n+2*, which are impossible for the the two first and twor last amino acids. Even the calculation statistics parameters aren't possible (cause of the ordinal placment, see more in details in the statistic part), it's preferable to eliminated the two first and two last fragments, wich reduce a little the time of calculation. More of that, the 'Z' not represent a trajectory, but the impossibility of calculate it (as NA or NaN represent the absence of value), so it's not macking sense to calculated statistics on thats fragments.

In [13]:
fasta_formed = fasta_transposed[2:-2]
print(fasta_formed)

[['o' 'o' 'l' ... 'a' 'd' 'b']
 ['p' 'p' 'c' ... 'f' 'j' 'f']
 ['a' 'a' 'd' ... 'k' 'k' 'k']
 ...
 ['d' 'd' 'd' ... 'd' 'd' 'd']
 ['d' 'd' 'd' ... 'd' 'd' 'd']
 ['d' 'd' 'd' ... 'd' 'd' 'd']]


At this point, we have a now a formated numpy array, wich contained a every trajectory for each fragments in rows. At this point, we can perform statistical analysis to determine the correlation between trajectories.



## III) Statistical analysis on trajectories

In the article of Pandini *et al.*, they use a statistical model to determined the correlation between local motions. In this aspect they determined a normalized mutual information, based on three different statistics information : the mutual information, the joint entropy and the expected error of the mutual information. Here, we implement the same approach to our strucural alphabet.

### 1) The mutual information

The first statistical parameter is the mutual information. In article of Pandini *et al* and in the GSA tools software, they calculated the mutual information *I(X;Y)* as the following equation:

$
I(X;Y)=\sum_{x\in X}\sum_{y\in Y} p(x,y)log_2(\frac{p(x,y)}{p_1(x) p_2(y)})
$

To implement that, we defined a fonction `calculate_MI` as:

```python
def calulate_MI(px, py, pxy):
    size = len(px)
        MI = 0
        for i in range(0,size):
            for j in range(0,size):
                if pxy[i,j] != 0:
                    MI += pxy[i,j] * math.log2(pxy[i,j]/(px[i]*py[j]))
        if MI > 0:
            return MI
        else:
            return 0 
```

### 2) The joint entropy

On the same approach, the joint entropy is defined as *H(X;Y)*:

$
H(X;Y)=\sum_{x\in X}\sum_{y\in Y} p(x,y)log_2 p(x,y)
$

```python
def calculate_joint_entropy(pxy):
    size = len(pxy)
    joint_entropy = 0
    for i in range(0,size):
        for j in range(0,size):
            if pxy [i,j] !=0:
                joint_entropy += pxy[i,j] * math.log2(pxy[i,j])
    if joint_entropy > 0:
        return joint_entropy
    else:
        return 0
```

### 3) The expected error of the mutual information

The third parameter needed for the normalized mutual information is the expected error. In fact, the calculation of the information theoretical quantities on a finite size sample is affected by random errors, inherent to the sample, which are negligeable in the case of Shanon's entropy, but affect the mutual information calculs. In this case, Roulston *et al.* () present a method to improve this calcul, which used the expect error on the MI. This error is calculated as:

$
\varepsilon (X;Y) = \frac{B^*_{XY} - B^*_X - B^*_Y +1}{2N}
$

Where $B^*_{XY}$ , $B^*_X$ , $B^*_Y$ are the count of probability different of zero in the stat *(X;Y)*, *X* and *Y*, respectively.

On this purpose, we defiened a fonction which calculate this expected error as following:

```python
def calculate_eeMI(bxy, bx, by, n):
    return (bxy - bx - by + 1)/(2 * n)
```


### 4) The normalized mutual information

After calculate all of this statistical data,we obtained three differents matrixes, which have the same lenght than the sequences of the protein in row and columns. Each matrixes indicate the mutual information, the joint entropy and the expected error for each fragment against each other fragment. Then, we can calculated the normalized mutual information, defined as following:

$ 
I^n_{LL}(C_i;C_j)=\frac{I(C_i;C_j) - \varepsilon(C_i;C_j)}{H(C_i;C_j)}
$

For calculate this, we do a one fonction which aim to do the mutual information normalized and every other part of the statistic analysis:

``` python
def calculate_MI_global(frames_all):
    seq_size = len(frames_all[:,0])
    weight = 1 / seq_size
    MI = np.zeros((seq_size,seq_size))
    joint_entropy = np.zeros((seq_size,seq_size))
    eeMI = np.zeros((seq_size,seq_size))
    normalized_MI = np.zeros((seq_size,seq_size))
    for i in range(0,len(frames_all)):
        for j in range(i,len(frames_all)):
            px = calculate_p(frames_all[i],weight) 
            py = calculate_p(frames_all[j],weight)
            pxy = calculate_pxy(frames_all[i],frames_all[j],weight)
            MI[i, j] = calulate_MI(px, py, pxy)
            joint_entropy[i, j] = calculate_joint_entropy(pxy)
            bx = np.count_nonzero(px)
            by = np.count_nonzero(py)
            bxy = np.count_nonzero(pxy)
            eeMI[i, j] = calculate_eeMI(bxy, bx, by, seq_size)
            if joint_entropy[i, j] != 0:
                normalized_MI[i, j] = (MI[i, j] - eeMI[i, j])/joint_entropy[i, j]
            else:
                normalized_MI[i, j] = 0
            if i != j:
                MI[j, i]=MI[i, j]
                eeMI[j, i]=eeMI[i, j]
                joint_entropy[j, i]=joint_entropy[i, j]
                normalized_MI[j, i] = normalized_MI[i, j]
    return MI, eeMI, joint_entropy, normalized_MI
```

In this fonction, only the numpy array of the fasta is needed to compute all of the model, and return the mutual information, the joint entropy, the mutual information and his expected error. every part of this analysis is wright in the `pb_analyze.py` file.

In [20]:
from pb_analyze import pba

MI, eeMI, joint_entropy, normalized_MI = pba.calculate_MI_global(fasta_formed)

ModuleNotFoundError: No module named 'pb_analyze'

In [ ]:
print(MI)

In [ ]:
print(eeMI)

In [ ]:
print(joint_entropy)

In [ ]:
print(normalized_MI)

## IV) Vizualisaton

In this last part, we add a graphic representation for matrixes with the use of `matplotlib` and `seaborn`.